# **Naver 쇼핑 수집한 내용 불러오기**
## **1 크롤링 수집함수**

In [1]:
# Web Crawler 함수
from urllib import parse
from urllib import parse, request

def crawler_naver(url):
    browser = "Mozilla/5.0 (X11; Linux i686) AppleWebKit/537.17 (KHTML, like Gecko) Chrome/24.0.1312.27 Safari/537.17"
    return request.urlopen(request.Request(url, headers = {'User-Agent':browser})).read().decode('utf8')

## **2 Loading 카테고리**
네이버 쇼핑 카테고리 목록 by **[Join inner list](https://stackoverflow.com/questions/716477/join-list-of-lists-in-python)**
```python
import itertools
a = [['a','b'], ['c']]
print(list(itertools.chain.from_iterable(a)))
```

In [2]:
import pandas as pd
import itertools
category_dict = pd.read_csv('./data/categoty_naver.csv').to_dict(orient='list')
category_dict = {k: [_ for _ in items if str(_) != 'nan']   for k, items in category_dict.items()}
category_values = list(itertools.chain.from_iterable(category_dict.values()))
print(f"total_length : {len(category_values)} 개")

total_length : 199 개


## **3 Maulo Item Result**
마을로 현재 Item 내용을 활용하여 수집한 내용
```python
# load and uncompress.
with gzip.open('./data/mauloItemNaver.pickle','rb') as f:
    items_maulo_category = pickle.load(f)
```

In [3]:
import pickle
with open('./data/category_Naver.pickle', 'rb') as f:
    items_maulo_category = pickle.load(f)
print("loading is done.")

loading is done.


In [4]:
%whos | grep list & str & dict

Variable               Type    Data/Info
----------------------------------------
category_dict          dict    n=10
category_values        list    n=199
items_maulo_category   dict    n=906


<br/>

# **Category Counting**
## **1 Category Matching & Counting**

In [5]:
items_count = {}
# Counting the matched items
for k,v in items_maulo_category.items():
    mathed = False
    for _ in v.split('>'):
        if _ in category_values:
            if _ not in list(items_count.keys()):
                items_count[_] = 1; matched = True
                #items_count[_] = [k]
            else:
                items_count[_] += 1; matched = True
                #items_count[_].append(k)
    if matched == False:
        print(k)
len(items_count)

70

## **2 Category Matching Token List**
Token list extracted

In [6]:
items_token_list = {}
# Counting the matched items
for k,v in items_maulo_category.items():
    mathed = False
    for _ in v.split('>'):
        if _ in category_values:
            if _ not in list(items_token_list.keys()):
                items_token_list[_] = [k]; matched = True
            else:
                items_token_list[_].append(k); matched = True
    if matched == False:
        print(k)

# "수산" Key Items List
">> ".join(items_token_list['수산'])

'주먹밥연어>> 굴비*소>> 굴비*대>> 냉동오징어>> 간고등어>> 임연수>> 알탕용(알&곤이)>> 삼치>> 꽁치>> 바지락살>> 날치알>> 생홍합살>> 황토우렁이>> 명태코다리>> 대구살>> 대구포>> 대구전>> 동태포>> 바로먹는고등어오븐구이>> 얼린오징어*썬것>> 가재새우살>> 연어스테이크>> 훈제연어>> 오징어젓>> 명란젓>> 참새우젓>> 새우육젓>> 황태포>> 황태채>> 국물멸치>> 천일염멸치(지리)>> 죽염멸치(중멸)>> 죽염디포리>> 쌈다시마>> 해초무침>> 해풍돌미역>> 태양건조참다시마>> 파래>> 참돌미역귀>> 생톳>> 생돌김자반>> 올리브파래김>> 구운김>> 김자반볶음>> 재래김>> 자른미역>> 미역줄기>> 다시마채>> 냉채용미역>> 곱창돌김>> 마른오징어>> 건홍새우>> 건미역>> 다짐대게살>> 다짐전복>> 덕수건어물>> 북어채>> 자멸+돌김 선물세트>> 중멸+돌김 선물세트>> 대멸+돌김 선물세트>> 영광 굴비세트 2호(전화문의)>> 영광 굴비세트 1호>> 홍진미채>> 건새우>> 현미영양식-된장미역>> 연하도움식-흰살생선>> 연하도움식-된장미역'

In [11]:
items_token_list['모니터주변기기']

['정립전자']

## **3 ReNewal the Category Table**
matching the table

- [dict result to Pandas Dataframe](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.from_dict.html)

In [7]:
result = {}
for k, v_list in category_dict.items():
    v_list_new = []
    marker = False
    # shopping item list loop
    for _key in v_list:
        # matching the count result roop
        for _k, _v in items_count.items():
            if _k == _key:  _key = f"({_v}){_key}" # Adding the count number
            else:        pass
        v_list_new.append(_key)
    result[k] = v_list_new

">> ".join(result["식품"])

'(44)축산>> (64)건강식품>> (67)수산>> (93)농산물>> (15)반찬>> (16)김치>> (59)음료>> (184)가공식품>> (1)다이어트식품>> (23)과자>> 아이스크림/빙수>> (36)냉동/간편조리식품>> 전통주>> 쿠킹박스'

In [8]:
# dict result to Pandas DataFrame Table
result_df = pd.DataFrame.from_dict(result, orient='index').T.fillna('')
result_df.to_csv('./data/categoty_naver_count.csv', index=None)
result_df.head()

,패션의류,패션잡화,화장품/미용,디지털/가전,가구/인테리어,출산/육아,식품,스포츠/레저,생활/건강,여가/생활편의
0,(1)여성의류,여성신발,(10)스킨케어,휴대폰,침실가구,분유,(44)축산,등산,공구,(2)공연/티켓
1,여성언더웨어/잠옷,(2)남성신발,선케어,휴대폰 액세서리,거실가구,기저귀,(64)건강식품,(1)캠핑,(16)문구/사무용품,원데이클래스
2,남성의류,(1)지갑,(5)클렌징,카메라/캠코더용품,주방가구,(2)물티슈,(67)수산,(2)수영,화방용품,예체능레슨
3,남성언더웨어/잠옷,여행용가방/소품,베이스메이크업,광학기기/용품,수납가구,(2)이유식,(93)농산물,스포츠액세서리,(2)관상어용품,자기계발/취미 레슨
4,,(1)신발용품,(7)색조메이크업,영상가전,아동/주니어가구,유모차,(15)반찬,골프,(7)수집품,(10)생활편의
